In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-21 11:21:25--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-21 11:21:25--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   152MB/s    in 0.5s    

2022-04-21 11:21:26 (152 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
psi4.set_output_file('log.txt')

psi4.set_memory('10GB')
psi4.set_num_threads(2)

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = Water
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''
psi4.set_options({'pcm': True})
psi4.pcm_helper(pcm)

#level = 'B3LYP-D3BJ/STO-3G' # Mulliken charge
#level = 'B3LYP/6-31G(d)'
#level = 'B3LYP/6-31G(d,p)' # Cation
#level = 'B3LYP/6-31+G(d)' # Anions and excited states
#level = 'B3LYP/6-31++G(d)' # Hydride
#level = 'B3LYP/6-311G(d)' # Fluorine
#level = 'MP2/6-31+G(d,p)'
#level = 'wb97x-d/cc-pvtz'
#level = 'M06-D3/6-311G(d,p)' # metal
level = 'M06-2X-D3/6-311G(d,p)' # non-metal
#level = 'B97-D3BJ/6-311G(d,p)'
#level = 'B3LYP-D3BJ/6-311G(d,p)'
#level = 'CAM-B3LYP/cc-pvtz'

scf_e, wfn = psi4.energy(level, return_wfn = True)
print('energy at the {} level of theory:\t{:.4f}'.format(level, scf_e))

energy at the M06-2X-D3/6-311G(d,p) level of theory:	-76.4164


In [4]:
psi4.oeprop(wfn, 'WIBERG_LOWDIN_INDICES')
!cat log.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
  PCM does not make use of molecular symmetry: further calculations in C1 point group.

*** tstart() called on 0160be7ee2ce
*** at Thu Apr 21 11:25:17 2022

   => Loading Basis Set <=

    Name: 6-311G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   151 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 2-3 entry H          line    32 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 

   => M06-2X-D3: Empirical Dispersion <=

    Grimme's -D3 (zero-damping) Dispersion Correction
    Grimme S.; Antony J.; Ehrlich S.; Krieg H. (2010), J. Chem. Phys., 132: 154104
    Parametrisation from:    L. Goerigk, S. Grimme, Phys. Chem. Chem. Phys. 13, 6670-6688, 2011

        s6 =       1.000000
        s8 =       0.000000
       sr6 =       1.619000
    alpha6 =      14.000000
       sr8 =       1.000000


         --------

In [5]:
import numpy as np
psi4.oeprop(wfn,'DIPOLE')
dipole_vec = psi4.variable('scf dipole')
dipole = np.sqrt(np.sum(dipole_vec ** 2))
print(f'{dipole:.3f} [D]')

0.908 [D]


In [6]:
level = 'B3LYP-D3BJ/STO-3G' # Mulliken charge
scf_e, wfn = psi4.energy(level, return_wfn = True)

In [7]:
import pandas as pd
df = pd.DataFrame()
psi4.oeprop(wfn, 'MULLIKEN_CHARGES')
df[level] = np.array(wfn.atomic_point_charges())
print (df)

   B3LYP-D3BJ/STO-3G
0          -0.353953
1           0.176977
2           0.176977
